# 그룹연산

pandas 의 그룹 연산은 데이터를 집계하거나 변환하는 등의 작업을 한번에 처리할 수 있는 강력한 기능이다.

- 분할 - 어떤 기준으로 데이터를 나누는 것
- 반영 - 함수 등을 적용하여 데이터를 처리하는 것
- 결합 - 처리한 결과를 다시 합치는 것.

이 과정을 하나로 묶어 분할-반영-결합(Split-Apply-Combine) 이라고 한다.

## 데이터 집계 - groupby

In [3]:
# groupy 로 평균값 구하기

import pandas as pd
df = pd.read_csv('../data/gapminder.tsv', sep='\t')
df.head()

,country,continent,year,lifeExp,pop,gdpPercap
0,Afghanistan,Asia,1952,28.801,8425333,779.445314
1,Afghanistan,Asia,1957,30.332,9240934,820.853030
2,Afghanistan,Asia,1962,31.997,10267083,853.100710
3,Afghanistan,Asia,1967,34.020,11537966,836.197138
4,Afghanistan,Asia,1972,36.088,13079460,739.981106


In [4]:
avg_life_exp_by_year = df.groupby('year').lifeExp.mean()
avg_life_exp_by_year

year
1952    49.057620
1957    51.507401
1962    53.609249
1967    55.678290
1972    57.647386
1977    59.570157
1982    61.533197
1987    63.212613
1992    64.160338
1997    65.014676
2002    65.694923
2007    67.007423
Name: lifeExp, dtype: float64

### 분할-반영-결합 과정 살펴보기

In [5]:
# 1. 연도별로 데이터를 나누는 과정
years = df.year.unique()
years

array([1952, 1957, 1962, 1967, 1972, 1977, 1982, 1987, 1992, 1997, 2002,
       2007], dtype=int64)

In [6]:
# 2. 연도별 평균값을 구한다. 반영 작업
y1952 = df.loc[df.year == 1952, :]
y1952.head()

,country,continent,year,lifeExp,pop,gdpPercap
0,Afghanistan,Asia,1952,28.801,8425333,779.445314
12,Albania,Europe,1952,55.230,1282697,1601.056136
24,Algeria,Africa,1952,43.077,9279525,2449.008185
36,Angola,Africa,1952,30.015,4232095,3520.610273
48,Argentina,Americas,1952,62.485,17876956,5911.315053


In [7]:
# 3. 1952 년의 데이터에서 lifeExp 의 평균값을 구한다. 반영 작업
y1952_mean = y1952.lifeExp.mean()
y1952_mean

49.057619718309866

In [8]:
# 4. 위의 작업을 반복한다.

y1957 = df.loc[df.year == 1957, :]
y1957_mean = y1957.lifeExp.mean()

y2007 = df.loc[df.year == 2007, :]
y2007_mean = y2007.lifeExp.mean()

In [9]:
# 5. 연도별로 계산한 lifeExp의 평균을 합친다. 결합
df2 = pd.DataFrame({"year": [1952, 1957, 2007], "":[y1952_mean, y1957_mean, y2007_mean]})
df2

,,year
0,49.057620,1952
1,51.507401,1957
2,67.007423,2007


### groupby 와 사용하는 집계 메서드

- count: 누락값을 제외한 데이터 수를 반환
- size: 누락값을 포함한 데이터 수를 반환
- mean: 평균값 반환
- std: 표준편차 반환
- min: 최소값 반환
- quantile(q=0.25): 백분위수 25%
- quantile(q=0.20): 백분위수 50%
- quantile(q=0.75): 백분위수 75%
- max: 최댓값 반환
- sum: 전체 합 반환
- var 분산 반환
- sem: 평균의 표준편차 반환
- describe: 데이터 수, 평균, 표준편차, 최소값, 백분위수(25, 50, 75%), 최대값을 모두 반환
- first: 첫 번째 행 반환
- last: 마지막 행 반환
- nth: n번째 행 반환

### agg 메서드로 사용자 함수와 groupby 메서드 조합하기
library 에서 제공하는 집계 메서드로 원하는 값을 계산할 수 없는 경우에는 직접 함수를 만들어 사용해야한다.
이번에는 사용자 함수와 groupby 메서드를 조합하여 사용한다.

In [10]:
# 평균값을 구하는 사용자 함수와 groupby 메서드
def my_mean(values):
    n = len(values)
    sum = 0
    for value in values:
        sum += value
    
    return sum / n

In [11]:
# agg 메서드를 사용하여 groupby 와 조합한다.

agg_my_mean = df.groupby('year').lifeExp.agg(my_mean)
agg_my_mean

year
1952    49.057620
1957    51.507401
1962    53.609249
1967    55.678290
1972    57.647386
1977    59.570157
1982    61.533197
1987    63.212613
1992    64.160338
1997    65.014676
2002    65.694923
2007    67.007423
Name: lifeExp, dtype: float64

In [12]:
# 2개의 인자값을 받아 처리하는 사용자 함수와 groupby 메서드
def my_mean_diff(values, diff_value):
    n = len(values)
    sum = 0
    for value in values:
        sum += value
    mean = sum / n
    return mean - diff_value    

In [13]:
global_mean = df.lifeExp.mean()
global_mean

59.474439366197174

In [14]:
agg_mean_diff = df.groupby('year').lifeExp.agg(my_mean_diff, diff_value=global_mean)
agg_mean_diff

year
1952   -10.416820
1957    -7.967038
1962    -5.865190
1967    -3.796150
1972    -1.827053
1977     0.095718
1982     2.058758
1987     3.738173
1992     4.685899
1997     5.540237
2002     6.220483
2007     7.532983
Name: lifeExp, dtype: float64

#### 여러개의 집계 메서드 한번에 사용하기

In [15]:
# 연도별로 그룹화한 lifeExp 열의 0이 아닌값의 개수, 평균, 표준편차를 한번에 계산하여 출력
import numpy as np
gdf = df.groupby('year').lifeExp.agg([np.count_nonzero, np.mean, np.std])
gdf

,count_nonzero,mean,std
year,,,
1952,142.0,49.057620,12.225956
1957,142.0,51.507401,12.231286
1962,142.0,53.609249,12.097245
1967,142.0,55.678290,11.718858
1972,142.0,57.647386,11.381953
1977,142.0,59.570157,11.227229
1982,142.0,61.533197,10.770618
1987,142.0,63.212613,10.556285
1992,142.0,64.160338,11.227380


In [16]:
# 집계 메서드를 딕셔너리에 담아 agg 메서드에 전달
# 딕셔너리 키로 집계 메서드를 적용할 열 이름을 전달하고 딕셔너리 값으로 집계 메서드를 전달한다.
gdf_dict = df.groupby('year').agg({'lifeExp': 'mean', 'pop': 'median', 'gdpPercap': 'median'})
gdf_dict

,gdpPercap,pop,lifeExp
year,,,
1952,1968.528344,3943953.0,49.057620
1957,2173.220291,4282942.0,51.507401
1962,2335.439533,4686039.5,53.609249
1967,2678.334741,5170175.5,55.678290
1972,3339.129407,5877996.5,57.647386
1977,3798.609244,6404036.5,59.570157
1982,4216.228428,7007320.0,61.533197
1987,4280.300366,7774861.5,63.212613
1992,4386.085502,8688686.5,64.160338


## 데이터 변환

### 표준점수 - 데이터의 평균과 표준편차의 차이

In [17]:
# 표준점수 계산하기
def my_zscore(x):
    return (x-x.mean()) / x.std()

In [18]:
# 연도별 lifeExp 열의 표준점수 계산
transform_z = df.groupby('year').lifeExp.transform(my_zscore)
transform_z.head()

0   -1.656854
1   -1.731249
2   -1.786543
3   -1.848157
4   -1.894173
Name: lifeExp, dtype: float64

In [19]:
# my_zscore 함수는 데이터를 표준화할 뿐 집계는 하지 않는다.
print(df.shape)
print(transform_z.shape)

(1704, 6)
(1704,)


### 누락값을 평균값으로 처리하기

In [20]:
# tips 데이터 집합에서 10개 행의 데이터를 가져온 후 
# total_bill 열의 값 4개를 임의로 선택하여 누락값으로 바꾼다.
import seaborn as sns
import numpy as np

np.random.seed(42)
tips_10 = sns.load_dataset('tips').sample(10)
tips_10.loc[np.random.permutation(tips_10.index)[:4], 'total_bill'] = np.NaN
tips_10

,total_bill,tip,sex,smoker,day,time,size
176,NaN,3.18,Male,No,Sat,Dinner,2
211,NaN,2.00,Male,No,Sun,Dinner,2
198,NaN,2.00,Male,No,Sun,Dinner,4
153,NaN,5.16,Male,Yes,Sat,Dinner,4
101,13.00,2.00,Female,Yes,Thur,Lunch,2
6,17.89,2.00,Male,Yes,Sun,Dinner,2
192,28.44,2.56,Male,Yes,Thur,Lunch,2
124,12.48,2.52,Female,No,Thur,Lunch,2
24,14.78,3.23,Male,No,Sun,Dinner,2
9,15.38,3.00,Female,Yes,Fri,Dinner,2


In [21]:
# 성별로 그룹화한 다음 각열의 데이터 수를 구한다.
count_sex = tips_10.groupby('sex').count()
count_sex

,total_bill,tip,smoker,day,time,size
sex,,,,,,
Male,3,7,7,7,7,7
Female,3,3,3,3,3,3


In [22]:
# 성별을 구분하여 total_bill 열의 데이터를 받아 평균값을 구한다.
def fill_na_mean(x):
    avg = x.mean()
    return x.fillna(avg)

In [23]:
# 성별을 구분한 total_bill 열의 데이터를 fill_na_mean 함수에 전달하여 평균을 구한 다음 
# tips_10에 새로운 열로 추가한다.
total_bill_group_mean = tips_10.groupby('sex').total_bill.transform(fill_na_mean)
total_bill_group_mean

176    20.37
211    20.37
198    20.37
153    20.37
101    13.00
6      17.89
192    28.44
124    12.48
24     14.78
9      15.38
Name: total_bill, dtype: float64

In [24]:
tips_10['fill_total_bill'] = total_bill_group_mean
tips_10

,total_bill,tip,sex,smoker,day,time,size,fill_total_bill
176,NaN,3.18,Male,No,Sat,Dinner,2,20.37
211,NaN,2.00,Male,No,Sun,Dinner,2,20.37
198,NaN,2.00,Male,No,Sun,Dinner,4,20.37
153,NaN,5.16,Male,Yes,Sat,Dinner,4,20.37
101,13.00,2.00,Female,Yes,Thur,Lunch,2,13.00
6,17.89,2.00,Male,Yes,Sun,Dinner,2,17.89
192,28.44,2.56,Male,Yes,Thur,Lunch,2,28.44
124,12.48,2.52,Female,No,Thur,Lunch,2,12.48
24,14.78,3.23,Male,No,Sun,Dinner,2,14.78
9,15.38,3.00,Female,Yes,Fri,Dinner,2,15.38


### 데이터 필터링

In [25]:
tips = sns.load_dataset('tips')
tips.shape

(244, 7)

In [26]:
tips['size'].value_counts()

2    156
3     38
4     37
5      5
6      4
1      4
Name: size, dtype: int64

In [27]:
tips_filtered = tips.groupby('size').filter(lambda x: x['size'].count() >= 30)
tips_filtered.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [28]:
tips_filtered['size'].value_counts()

2    156
3     38
4     37
Name: size, dtype: int64

### 그룹 오브젝트

In [29]:
# groupby 메서드의 결과값인 그룹 오브젝트를 살펴본다

In [31]:
# tips 데이터에서 임의로 10개의 데이터 추출
tips_10 = sns.load_dataset('tips').sample(10, random_state=42)
tips_10

,total_bill,tip,sex,smoker,day,time,size
24,19.82,3.18,Male,No,Sat,Dinner,2
6,8.77,2.00,Male,No,Sun,Dinner,2
153,24.55,2.00,Male,No,Sun,Dinner,4
211,25.89,5.16,Male,Yes,Sat,Dinner,4
198,13.00,2.00,Female,Yes,Thur,Lunch,2
176,17.89,2.00,Male,Yes,Sun,Dinner,2
192,28.44,2.56,Male,Yes,Thur,Lunch,2
124,12.48,2.52,Female,No,Thur,Lunch,2
9,14.78,3.23,Male,No,Sun,Dinner,2
101,15.38,3.00,Female,Yes,Fri,Dinner,2


In [32]:
# groupby 메서의 결과값이 group object 라는것을 확인한다.

grouped = tips_10.groupby('sex')
grouped

In [33]:
# group object에 포함된 그룹을 보려면 groups 속성을 출력한다.
grouped.groups

{'Female': Int64Index([198, 124, 101], dtype='int64'),
 'Male': Int64Index([24, 6, 153, 211, 176, 192, 9], dtype='int64')}

In [34]:
# 그룹 오브젝트 평균 구하기

In [35]:
# python 은 자동으로 계산할 수 있는 열을 골라 준다.

avgs = grouped.mean()
avgs

,total_bill,tip,size
sex,,,
Male,20.02,2.875714,2.571429
Female,13.62,2.506667,2.000000


In [37]:
tips_10.columns

Index(['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size'], dtype='object')

### 그룹 오브젝트 활용

In [38]:
# 그룹 오브젝트에서 데이터 추출하고 반복

In [40]:
# get_group 메서드를 사용하여 성별이 여성인 데이터만 추출한다.
female = grouped.get_group('Female')
female

,total_bill,tip,sex,smoker,day,time,size
198,13.00,2.00,Female,Yes,Thur,Lunch,2
124,12.48,2.52,Female,No,Thur,Lunch,2
101,15.38,3.00,Female,Yes,Fri,Dinner,2


In [42]:
# 그룹 오브젝트를 반복문에 사용: 결과는 튜플이다.
for group in grouped:
    print(group)

('Male',      total_bill   tip   sex smoker   day    time  size
24        19.82  3.18  Male     No   Sat  Dinner     2
6          8.77  2.00  Male     No   Sun  Dinner     2
153       24.55  2.00  Male     No   Sun  Dinner     4
211       25.89  5.16  Male    Yes   Sat  Dinner     4
176       17.89  2.00  Male    Yes   Sun  Dinner     2
192       28.44  2.56  Male    Yes  Thur   Lunch     2
9         14.78  3.23  Male     No   Sun  Dinner     2)
('Female',      total_bill   tip     sex smoker   day    time  size
198       13.00  2.00  Female    Yes  Thur   Lunch     2
124       12.48  2.52  Female     No  Thur   Lunch     2
101       15.38  3.00  Female    Yes   Fri  Dinner     2)


In [43]:
# 그룹 오브젝트 계산하고 살펴보기
bill_sex_time = tips_10.groupby(['sex', 'time'])
group_avg = bill_sex_time.mean()

group_avg

total_bill       tip      size
sex    time                                  
Male   Lunch    28.440000  2.560000  2.000000
       Dinner   18.616667  2.928333  2.666667
Female Lunch    12.740000  2.260000  2.000000
       Dinner   15.380000  3.000000  2.000000

In [44]:
type(group_avg)

pandas.core.frame.DataFrame

In [45]:
group_avg.columns

Index(['total_bill', 'tip', 'size'], dtype='object')

In [46]:
group_avg.index

MultiIndex(levels=[['Male', 'Female'], ['Lunch', 'Dinner']],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]],
           names=['sex', 'time'])

In [48]:
# MultiIndex인 경우 reset_index 메서드를 사용하여 인덱스를 새로 부여할 수 있다.
group_method = tips_10.groupby(['sex', 'time']).mean().reset_index()
group_method

,sex,time,total_bill,tip,size
0,Male,Lunch,28.440000,2.560000,2.000000
1,Male,Dinner,18.616667,2.928333,2.666667
2,Female,Lunch,12.740000,2.260000,2.000000
3,Female,Dinner,15.380000,3.000000,2.000000
